In [3]:
import pandas as pd 
import numpy as np 

import psycopg2 as pg2
import sqlalchemy
import s3fs

import julian
import datetime
import calendar
import time

import gdal
import os
import csv

import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline

import src.humble_beginnings as helper

In [4]:
lat_long_list = helper.get_meta_data()

In [36]:
metadata,idi=helper.load_metadata(meta_data_path='/Users/Berzyy/projects/plant_forecast/preloaded_data/ghcnd-stations.txt')

In [45]:
idi['BDM00078016']

array([32.3667, -64.6833, 6.1, 'L'], dtype=object)

In [5]:
station_id = list( x[0] for x in lat_long_list )
lat= list( x[1] for x in lat_long_list )
long= list( x[2] for x in lat_long_list )

In [46]:
def cluster_wd(name='weather', loc='localhost'):
    """Get the Weather Data for all stations for one day
    """
    conn = pg2.connect(dbname=name, host=loc)
    cur = conn.cursor()
    conn.autocommit = True

    get_weather_command = f"""SELECT station_id 
                            FROM station_metadata 
                            WHERE ST_Contains(ST_GeomFromText('POLYGON((-140 60, -50 60, -50 20, -40 20, -140 60))',4326),geom);"""

    cur.execute(get_weather_command)
    weather = cur.fetchall()
    stations = tuple(map(' '.join, weather))
    #df = helper.prepare_weather_data_for_merge(weather)
    #df = helper.pivot_weather_data_frame(df)
    conn.close()
    return stations
    

In [64]:
def make_a_matrix(idi, m_df, d_df, stations):
    """Takes in dictionary, a metadata df , and a data df
    """
    for station in stations:
        latitude = idi[station][0]
        longitude = idi[station][1]
        elevation = idi[station][2]
        

In [65]:
make_a_matrix(idi,metadata, df,stations)

('BDM00078016',
 'BDW00013602',
 'CA001066420',
 'CA001070571',
 'CA001070572',
 'CA001070573',
 'CA001071092',
 'CA001072075',
 'CA001072902',
 'CA001073040',
 'CA001073278',
 'CA001073347',
 'CA001073349',
 'CA00107335J',
 'CA001073L4F',
 'CA001074276',
 'CA001074328',
 'CA001074331',
 'CA001075202',
 'CA001075252',
 'CA001075253',
 'CA001075520',
 'CA001078209',
 'CA001078L3D',
 'CA00107G879',
 'CA00107GHG9',
 'CA001090300',
 'CA001090586',
 'CA001090660',
 'CA001090R0R',
 'CA001091512',
 'CA001092120',
 'CA001092519',
 'CA001092520',
 'CA001092578',
 'CA001092579',
 'CA001092678',
 'CA001092685',
 'CA001092904',
 'CA001092905',
 'CA001092970',
 'CA001092972',
 'CA001093165',
 'CA001093474',
 'CA001093475',
 'CA001093680',
 'CA001093777',
 'CA001093920',
 'CA001094602',
 'CA001094785',
 'CA001094897',
 'CA001094948',
 'CA001094950',
 'CA001094955',
 'CA001094962',
 'CA001095000',
 'CA0010952B9',
 'CA001095410',
 'CA001095412',
 'CA001095617',
 'CA001095740',
 'CA001096435',
 'CA0010

In [49]:
stations = cluster_wd()

In [24]:
groups = df.groupby(df.index)

In [26]:
groups.head()

,measurement_type,PRCP,SNOW,SNWD,TMAX,TMIN
station_id,measurement_date,,,,,
ARM00087582,2000-01-01,NaN,NaN,NaN,NaN,200.0
ASN00009661,2000-01-01,0.0,NaN,NaN,NaN,NaN
ASN00014227,2000-01-01,462.0,NaN,NaN,NaN,NaN
ASN00014814,2000-01-01,98.0,NaN,NaN,NaN,NaN
ASN00015643,2000-01-01,0.0,NaN,NaN,350.0,NaN
ASN00033110,2000-01-01,0.0,NaN,NaN,NaN,NaN
ASN00033211,2000-01-01,0.0,NaN,NaN,NaN,NaN
ASN00036096,2000-01-01,0.0,NaN,NaN,NaN,NaN
ASN00040209,2000-01-01,0.0,NaN,NaN,290.0,202.0
